In [15]:
from ipyleaflet import *
from ipyleaflet import FullScreenControl
from ipywidgets import HTML, IntSlider, SelectionSlider, Dropdown, Button
from shapely.geometry import shape, Point
from ipywidgets import Layout
from datetime import datetime
import os

import requests
from datetime import datetime

import pickle
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsRegressor


In [2]:
api_key = os.getenv("GOOGLE_MAPS_API_KEY")

if api_key is None:
    print("Error: No API key found. Please set the environment variable.")
else:
    print("API key loaded successfully.")

API key loaded successfully.


In [3]:
# loading the models
loadedModel = pickle.load(open("./taxi_knr_240k.sav", 'rb'))
loadedScaler = pickle.load(open("./taxi_scaler_240k.sav", 'rb'))

In [4]:
def getTaxiTripLength(longIn, latIn, longOut, latOut, month, day, hour) -> float: # returns the estimated trip length in seconds
    scaledX = loadedScaler.transform(np.array([latIn, latOut, longIn, longOut,month,day,hour]).reshape(1, -1))  
    return loadedModel.predict(scaledX)[0][0]

In [5]:
def get_route_data(origin, destination, api_key, departure_time=None, mode="walking"):
    """
    Make an API call to Google Maps Directions API to get route data.
    
    Args:
    - origin (tuple): A tuple containing latitude and longitude of the origin point.
    - destination (tuple): A tuple containing latitude and longitude of the destination point.
    - api_key (str): Your Google Maps API key.
    
    Returns:
    - dict: A dictionary containing the full response from the Google Maps API.
    """
    # Directions API endpoint URL
    url = "https://maps.googleapis.com/maps/api/directions/json"

    if isinstance(departure_time, datetime):
        
        current_time = datetime.now()
        if departure_time <= current_time:
            departure_time = "now"
        else:
            departure_time = int(departure_time.timestamp())
    elif departure_time is None:
        # Set departure_time to 'now' if not provided
        departure_time = "now"
    
    # Define the parameters for the request
    params = {
        "origin": f"{origin[0]},{origin[1]}",
        "destination": f"{destination[0]},{destination[1]}",
        "mode": mode,
        "key": api_key,
        "departure_time": f"{departure_time}"
    }
    
    # Make the GET request to Google Maps API
    response = requests.get(url, params=params)
    data = response.json()

    # Check the status and return the data if the status is "OK"
    if data["status"] == "OK":
        return data
    else:
        # Handle potential errors in the API response
        print(f"Error on get_route_data: {data['status']}")
        return None

In [6]:
def extract_deplacement_time(route_data):
    """
    Extract deplacement time from the route data obtained from Google Maps API.
    
    Args:
    - route_data (dict): A dictionary containing the response from Google Maps API.
    
    Returns:
    - (str, int): A tuple containing walking time in human-readable format and duration in seconds.
    """
    if route_data is None:
        print("Error: No route data available to extract walking time.")
        return None, None

    # Extract duration information from route data
    try:
        route = route_data["routes"][0]
        leg = route["legs"][0]
        deplacement_time = leg["duration"]["text"]  # Text of the duration
        deplacement_time_seconds = leg["duration"]["value"]  # Duration in seconds

        return deplacement_time, deplacement_time_seconds
    except (KeyError, IndexError) as e:
        print(f"Error extracting walking time: {e}")
        return None, None

In [7]:
def extract_addresses(route_data):
    """
    Extract the start and destination addresses from the route data obtained from Google Maps API.
    
    Args:
    - route_data (dict): A dictionary containing the response from Google Maps API.
    
    Returns:
    - (str, str): A tuple containing the start address and destination address.
    """
    if route_data is None:
        print("Error: No route data available to extract addresses.")
        return None, None

    try:
        # Access the legs information in the route
        leg = route_data["routes"][0]["legs"][0]

        # Extract the start and end addresses
        start_address = leg["start_address"]  # Start address in human-readable format
        end_address = leg["end_address"]      # End address in human-readable format

        return start_address, end_address
    except (KeyError, IndexError) as e:
        print(f"Error extracting addresses: {e}")
        return None, None

In [8]:
def extract_polyline(route_data):
    """
    Extract the encoded polyline from the Google Maps Directions API route data.
    
    Args:
    - route_data (dict): The full response data from the Google Maps Directions API.
    
    Returns:
    - str: The encoded polyline if available, None otherwise.
    """
    try:
        # Extract the overview polyline from the route
        polyline = route_data["routes"][0]["overview_polyline"]["points"]
        return polyline
    except (KeyError, IndexError):
        print("Error: Unable to extract polyline from the provided route data.")
        return None

In [9]:
import polyline

def decode_polyline(encoded_polyline):
    """
    Decodes an encoded polyline string into latitude/longitude pairs.

    Args:
    - encoded_polyline (str): The encoded polyline string to decode.

    Returns:
    - list: A list of latitude/longitude pairs (tuples).
    """
    if not encoded_polyline:
        print("No encoded polyline provided.")
        return None
    
    try:
        # Decode the polyline into a list of lat/lon tuples
        decoded_points = polyline.decode(encoded_polyline)
        return decoded_points
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [10]:
# load GeoJSON file containing sectors
with open('manhattan.geojson') as f:
    js = json.load(f)

In [11]:
def checkIfIn(cords:list):
    point=Point(cords[1],cords[0])
    # check each polygon to see if it contains the point
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return True
    return False

In [12]:
def format_time(seconds):
    if seconds < 60:
        # Only seconds
        return f"{int(seconds)} seconds"
    elif seconds < 3600:
        # Only minutes
        minutes = seconds // 60
        return f"{int(minutes)} minutes"
    else:
        # Hours and minutes
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        if minutes == 0:
            return f"{hours} hours"
        else:
            return f"{hours} hours and {minutes} minutes"

In [22]:
m = Map(center=(40.7128, -74.0060), zoom=15,layout=Layout(height='1000px'),scroll_wheel_zoom=True, fullscreen=True)

def random_color(feature):
    return {
        'color': 'purple',
        'fillColor': 'green',
    }

geo_json = GeoJSON(
    data=js,
    style={
        'opacity': 1, 'dashArray': '40', 'fillOpacity': 0.1, 'weight': 1
    },
    style_callback=random_color
)


m.add(FullScreenControl())
m.add(geo_json)
now = datetime.now()

click_pos=[]
markers=[]
polyline_layer=[]

time_wigdet = HTML()
deplacement_time_control = WidgetControl(widget=time_wigdet, position='topright')

icon_start = AwesomeIcon(
    name='play',
    marker_color='green',
    icon_color='white',
    spin=False
)

icon_stop = AwesomeIcon(
    name='stop',
    marker_color='red',
    icon_color='black',
    spin=False
)

month_slider = SelectionSlider(
    options=[('January', 1), ('February', 2), ('March', 3), ('April', 4),
             ('May', 5), ('June', 6), ('July', 7), ('August', 8),
             ('September', 9), ('October', 10), ('November', 11), ('December', 12)],
    value=now.month,
    description='Month:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

day_slider = IntSlider(
    value=now.day,
    min=1,
    max=31,
    step=1,
    description='Day:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

hour_slider = IntSlider(
    value=now.hour,
    min=0,
    max=23,
    step=1,
    description='Hour:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

time_button = Button(description="Ok")

model_selector = Dropdown(
    # options=['walking', 'bicycling', 'transit', 'driving'],
    options=['walking', 'bicycling'],
    description='Mode:',
    disabled=False
)

month_days = {
    1: 31,  # January
    2: 28,  # February (non-leap year), or 29 (leap year)
    3: 31,  # March
    4: 30,  # April
    5: 31,  # May
    6: 30,  # June
    7: 31,  # July
    8: 31,  # August
    9: 30,  # September
    10: 31, # October
    11: 30, # November
    12: 31  # December
}

def update_day_slider(change):
    month = month_slider.value

    # If the month is February, be able to set up to 29 days here for leap years
    if month == 2:
        day_slider.max = 28
    else:
        day_slider.max = month_days[month]
    
    # Adjust the current value if it's larger than the new max
    if day_slider.value > day_slider.max:
        day_slider.value = day_slider.max

month_slider.observe(update_day_slider, names='value')
update_day_slider(None)

title = HTML("<h4 style='margin: 0px; margin-left: 10px'>Select Time</h4>")

time_control_box = Box([title, day_slider, month_slider, hour_slider, time_button, model_selector], layout={'display': 'flex', 'flex_flow': 'column', 'align_items': 'stretch'})
time_control = WidgetControl(widget=time_control_box, position='bottomright')

m.add_control(time_control)

def compute_travel_time(change):
    if len(click_pos)>=2:
        departure_time = datetime(2025, month_slider.value, day_slider.value, hour_slider.value)
        route_data = get_route_data(click_pos[0], click_pos[1], api_key, departure_time, model_selector.value)
        deplacement_time, deplacement_time_seconds = extract_deplacement_time(route_data)
        taxi_time = getTaxiTripLength(longIn=click_pos[0][0], latIn=click_pos[0][1], longOut=click_pos[1][0], latOut=click_pos[1][1], month=month_slider.value, day=day_slider.value, hour=hour_slider.value)

        time_wigdet.value = f"<b>Estimated {model_selector.value} time:</b> {format_time(deplacement_time_seconds)}<br><b>Taxi deplacement time:</b> {format_time(taxi_time)}"

        if polyline_layer is not None and len(polyline_layer)>0:
            m.remove_layer(polyline_layer[0])
            polyline_layer.pop(0)
            
        polyline_layer.append(Polyline(locations=decode_polyline(extract_polyline(route_data)), color="blue", fill=False))
        m.add_layer(polyline_layer[0])

def markers_handler(**kwargs):

    coords=kwargs.get('coordinates')
    if(checkIfIn(coords)):
        click_pos.append(coords)
    
        if len(click_pos)==3:
            m.remove_layer(markers[0])
            m.remove_layer(markers[1])

            click_pos.pop(0)
            markers.pop(0)

            marker_start=Marker(location=click_pos[0],icon=icon_start)
            marker_stop=Marker(location=click_pos[1],icon=icon_stop)
        
            m.add(marker_start)
            m.add(marker_stop)
            markers[0]=marker_start
            markers.append(marker_stop)

    
        elif len(click_pos)==2:
            marker_stop=Marker(location=click_pos[1],icon=icon_stop)
            m.add(marker_stop)
            markers.append(marker_stop)

            m.add_control(deplacement_time_control)
        else:
            marker_single=Marker(location=click_pos[0],icon=icon_start)
            m.add(marker_single)
            markers.append(marker_single)
    
        compute_travel_time(None)
    else:
        m.fit_bounds([[40.69625781921317, -74.02656555175783],[40.88834126500965, -73.89953613281251]])

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        markers_handler(**kwargs)


time_button.on_click(compute_travel_time)
model_selector.observe(compute_travel_time, names='value')

m.fit_bounds([[40.69625781921317, -74.02656555175783],[40.88834126500965, -73.89953613281251]])
m.on_interaction(handle_click)

m

Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

c:\Users\cemuelle\AppData\Local\pypoetry\Cache\virtualenvs\hackaton-yFR1hobE-py3.10\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


Error on get_route_data: ZERO_RESULTS
Error: No route data available to extract walking time.


c:\Users\cemuelle\AppData\Local\pypoetry\Cache\virtualenvs\hackaton-yFR1hobE-py3.10\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


TypeError: '<' not supported between instances of 'NoneType' and 'int'

...

TypeError: '<' not supported between instances of 'NoneType' and 'int'

Error on get_route_data: ZERO_RESULTS
Error: No route data available to extract walking time.


c:\Users\cemuelle\AppData\Local\pypoetry\Cache\virtualenvs\hackaton-yFR1hobE-py3.10\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


TypeError: '<' not supported between instances of 'NoneType' and 'int'

c:\Users\cemuelle\AppData\Local\pypoetry\Cache\virtualenvs\hackaton-yFR1hobE-py3.10\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\cemuelle\AppData\Local\pypoetry\Cache\virtualenvs\hackaton-yFR1hobE-py3.10\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
